In [1]:
import logging
from importlib import reload
reload(logging)
logger = logging.getLogger(__name__)

In [2]:
import pandas as pd
import time
import random
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from pprint import pprint
from pathlib import Path
import numpy as np
from llm_assessor import (
    run_prompt_chain,
    extract_marks_from_graded_answer,
)

load_dotenv(override=True)

ERROR:root:Failed to update token costs. Using static costs.
/Users/junaid/.virtualenvs/audenshaw_exam_validation/lib/python3.11/site-packages/tokencost/constants.py:61: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  TOKEN_COSTS = TOKEN_COSTS_STATIC


True

In [3]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

'/Users/junaid/Developer/audenshaw_exam_validation'

In [4]:
# Load Directories containing Prompt Templates and Student Answers
prompt_dir = Path("prompt_templates")

data_dir = Path(f"{workdir}/validation_results/processed_data")

In [5]:
# Load LLM
llm = ChatOpenAI(
    model_name="gpt-4o", 
    temperature=0.00, 
    max_tokens=800,
    )

In [6]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt="%Y-%m-%d %H:%M:%S",
    )


# Load in Student Answers

In [7]:
# Load in Data
completed_paper_df = pd.read_csv(data_dir / "student_answers_augmented.csv")
#completed_paper_df = completed_paper_df[(completed_paper_df.subject_id == "edexcel_business_studies") & (completed_paper_df.student_id == 5095)]
student_answers_records = completed_paper_df.to_dict(orient="records")

# Read in Prompt Templates

In [8]:
# Load all prompt templates
prompts = {
    "grade_answer": 
    {
        "aqa_history": {
            "hs_analyse": load_prompt(prompt_dir/ "aqa_history" / "analyse_prompt" / "grade_answer_aqa_history_analyse_prompt.json"),
            "hs_explain": load_prompt(prompt_dir/ "aqa_history" / "explain_prompt" / "grade_answer_aqa_history_explain_prompt.json"),
            "hs_judgement": load_prompt(prompt_dir/ "aqa_history" / "judgement_prompt" / "grade_answer_aqa_history_judgement_prompt.json"),
            "hs_spag": load_prompt(prompt_dir/ "aqa_history" / "spag_prompt" / "grade_answer_aqa_history_spag_prompt.json")
        },
        "edexcel_business_studies": {
            "bs_analyse": load_prompt(prompt_dir/ "edexcel_business_studies" / "analyse_prompt" / "grade_answer_edexcel_business_studies_analyse_prompt.json"),
            "bs_discuss": load_prompt(prompt_dir/ "edexcel_business_studies" / "discuss_prompt" / "grade_answer_edexcel_business_studies_discuss_prompt.json"),
            "bs_evaluate": load_prompt(prompt_dir/ "edexcel_business_studies" / "evaluate_prompt" / "grade_answer_edexcel_business_studies_evaluate_prompt.json"),
            "bs_explain": load_prompt(prompt_dir/ "edexcel_business_studies" / "explain_prompt" / "grade_answer_edexcel_business_studies_explain_prompt.json"),
            "bs_identify": load_prompt(prompt_dir/ "edexcel_business_studies" / "identify_prompt" / "grade_answer_edexcel_business_studies_identify_prompt.json"),
            "bs_justify": load_prompt(prompt_dir/ "edexcel_business_studies" / "justify_prompt" / "grade_answer_edexcel_business_studies_justify_prompt.json"),
            "bs_outline": load_prompt(prompt_dir/ "edexcel_business_studies" / "outline_prompt" / "grade_answer_edexcel_business_studies_outline_prompt.json"),
            "bs_state": load_prompt(prompt_dir/ "edexcel_business_studies" / "state_prompt" / "grade_answer_edexcel_business_studies_state_prompt.json"),
            "bs_mcq": load_prompt(prompt_dir/ "edexcel_business_studies" / "mcq_prompt" / "grade_answer_edexcel_business_studies_mcq_prompt.json"),
            "bs_calculate": load_prompt(prompt_dir/ "edexcel_business_studies" / "calculate_prompt" / "grade_answer_edexcel_business_studies_calculate_prompt.json")
        }
    },
    "extract_marks": load_prompt(prompt_dir / "extract_mark_count" / "extract_mark_count_prompt.json"),
    }

# Grade Answers for Each Student

## Pass 1: Grade all Student Answers with Few Shot Prompting

In [9]:
# Pass 1: Grade All Answers
for _, student_answer in enumerate(student_answers_records):
    
    student_answer["start_time"] = time.time()
    prompt_template = prompts["grade_answer"].get(student_answer["subject_id"]).get(student_answer["question_type"])
    
    if (student_answer["answer_text"] is np.nan or len(student_answer["answer_text"]) == 0):
        
        student_answer["llm_graded_answer"] = "No Answer is provided. Therefore 0 marks are awarded for this answer."   
        student_answer["llm_graded_answer_token_costing"] = 0
        student_answer["llm_awarded_marks"] = 0
        student_answer["llm_awarded_marks_token_costing"] = 0
    
    else:
        try:
            # Grade Answer
            logging.info(f"Starting Grading Answer for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")

            input_args = {
                "question": student_answer["question_text"], 
                "answer": student_answer["answer_text"], 
                "mark_scheme": student_answer["mark_scheme_text"], 
                "context": student_answer["context"]
                }
            prompt_template_input_args = dict((k, input_args[k]) for k in prompt_template.input_variables if k in input_args)

            graded_answer_response = run_prompt_chain(
                prompt_template=prompt_template.template,
                llm=llm,
                burn_in_runs=1,
                **prompt_template_input_args
                )
            
            student_answer["llm_graded_answer"] = graded_answer_response["prompt_chain_response"]
            student_answer["llm_graded_answer_token_costing"] = graded_answer_response["prompt_chain_token_costing"]
            
            logging.info(f"Completed Grading Answer for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")
            
            # Extract Marks
            logging.info(f"Starting Extracting Marks for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")
            
            extract_marks_response = run_prompt_chain(
                prompt_template="Extract the number of marks awarded to the student answer. Read the answer carefully and extract the final number of marks to be awarded to the student. Report the number of marks only. Answer: {answer}",
                llm=ChatOpenAI(model_name="gpt-4o", temperature=0.0, max_tokens=500),
                burn_in_runs=1,
                answer=student_answer["llm_graded_answer"]
                )
            marks = extract_marks_response["prompt_chain_response"]
            marking_token_cost = extract_marks_response["prompt_chain_token_costing"]

            student_answer["llm_awarded_marks"] = int(marks)
            student_answer["llm_awarded_marks_token_costing"] = marking_token_cost
            
            logging.info(f"Completed Extracting Marks for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")

        except:
            logging.info(f"Question type: {student_answer['question_type']} is not supported. Assigning NA and skipping to next question.")

            student_answer["llm_graded_answer"] = "NA"   
            student_answer["llm_graded_answer_token_costing"] = 0
            student_answer["llm_awarded_marks"] = np.nan
            student_answer["llm_awarded_marks_token_costing"] = 0
   
    # Compute hitrate
    min_mark = min( [student_answer.get("awarded_marks")] )
    max_mark = max( [student_answer.get("awarded_marks")] )

    if (student_answer["awarded_marks"] == 0 and student_answer["answer_text"] is np.nan):
        student_answer["llm_mark_hitrate"] = np.nan
    else:
        student_answer["llm_mark_hitrate"]=min_mark<=student_answer["llm_awarded_marks"]<=max_mark

    student_answer["end_time"] = time.time()
    student_answer["elapsed_time_in_seconds"] = student_answer["end_time"] - student_answer["start_time"]

    # Randomly sleep for seconds to avoid API throttling. Between 1-3 seconds
    seconds_to_sleep = random.sample([1,2,3], 1)[0]

    logging.info(f"Sleeping for {seconds_to_sleep} seconds to avoid API Throttling.")
    time.sleep(seconds_to_sleep)    

2024-07-24 21:27:40 - INFO - Starting Grading Answer for Student ID: 5000 and Question ID: 1.0
2024-07-24 21:27:40 - INFO - Construct LLM Chain
2024-07-24 21:27:40 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-07-24 21:27:46 - INFO - Completed Grading Answer for Student ID: 5000 and Question ID: 1.0
2024-07-24 21:27:46 - INFO - Starting Extracting Marks for Student ID: 5000 and Question ID: 1.0
2024-07-24 21:27:46 - INFO - Construct LLM Chain
2024-07-24 21:27:46 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-07-24 21:27:46 - INFO - Completed Extracting Marks for Student ID: 5000 and Question ID: 1.0
2024-07-24 21:27:46 - INFO - Sleeping for 3 seconds to avoid API Throttling.
2024-07-24 21:27:49 - INFO - Starting Grading Answer for Student ID: 5000 and Question ID: 2.0
2024-07-24 21:27:49 - INFO - Construct LLM Chain
2024-07-24 21:27:49 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-07-24 21:27:54 - INFO - Completed Grading Answer for Student ID: 5000 and Question ID: 2.0
2024

In [10]:
# Combine Student Answers and save as a DataFrame
completed_paper_df = pd.DataFrame(student_answers_records)
ordered_cols = [
    'subject_id', 'question_id', 'question_type', 'student_id',
    'question_text', 'mark_scheme_text', 'context', 'answer_text', 
    'llm_graded_answer', 'awarded_marks', 'llm_awarded_marks', 'total_marks',
    'llm_mark_hitrate', 'answer_id', 'linked_answer_id', 'topic_id', 
    'answer_scanned_image', 'elapsed_time_in_seconds', 'llm_graded_answer_token_costing', 'llm_awarded_marks_token_costing'
    ]

completed_paper_df = completed_paper_df[ ordered_cols]

In [ ]:
# student_answer = [i for i in student_answers_records if i["question_id"] == 3.4][0]
# student_answer

In [ ]:
# print("Mark Scheme:\n", student_answer["mark_scheme_text"])
# print("-"*40)
# print("Student Answer:\n", student_answer["answer_text"])
# print("-"*40)
# print("Graded Answer:\n", student_answer["llm_graded_answer"])
# print("-"*40)
# print("LLM Marks:\n", student_answer["llm_awarded_marks"])
# print("-"*40)
# print("Actual Marks:\n", student_answer["awarded_marks"])

In [11]:
# Save graded student answers
completed_paper_df.to_csv(data_dir / "student_answers_llm_graded.csv", index=False)